## Interactive Visual Analytics with Folium

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

In [ ]:
import folium
import pandas as pd

In [ ]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


#### Task 1: Mark all launch sites on a map

In [ ]:
# Download and read the `spacex_launch_geo.csv`
#from js import fetch
#import io

#URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
#resp = await fetch(URL)
#spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
#spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)    # Working in Google Colab where pandas df can be created directly with a URL
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
# Selecting relevant sub-columns: 'Launch Site', 'Lat (Latitude)', 'Long (Longitude)', 'class'
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index = False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
# Create a folium map object with NASA Johnson Space Center's coordinates as initial center location

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location = nasa_coordinate, zoom_start = 10)

In [ ]:
# Use folium.Circle to add a highlighted circle area with a text label on a specific coordinate

# 1. Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius = 1000, color = '#d35400', fill = True).add_child(folium.Popup('NASA Johnson Space Center'))

# 2. Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Crate an icon as a text label
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style = "font-size:12; color: #d35400;"><b>%s</b></div>'%'NASA JSC'
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

Add a circle for each launch site in dataframe <code>launch_sites</code>.

* Create and add <code>folium.Circle</code>
* Create and add <code>folium.Marker</code>

for each launch site on the site map.


In [ ]:
site_map = folium.Map(location = nasa_coordinate, zoom_start = 5)

for site, lat, lng in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
  circle = folium.Circle(location = [lat, lng], radius = 50, color = '#d35400', fill = True).add_child(folium.Popup(site))
  marker = folium.Marker(
      location = [lat, lng],
      icon = DivIcon(
          icon_size = (20, 20),
          icon_anchor = (0, 0),
          html = '<div style = "font-size:12; color: #d35400";><b>%s</b></div>'%'{}'.format(site)
      )
  )
  site_map.add_child(circle)
  site_map.add_child(marker)

site_map

In [ ]:
# Questions:
# Q1 Are all laucnh sites in close proximity to the equator line?
# Answer - No. "VAFB SLC-4E" is relatively farther from equator than the other three launch sites.

# Q2 Are all launch sites in very close proximity to the coast?
# Answer - Yes.

#### Task 2: Mark the success/failed launches for each site on the map.

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Create markers for all launch records - green for successfull launch (class = 1) and red for failed launch (class = 0).

Note that all launches took place in one of the four launch sites, which means many launch records will have the exact same coordinate. Use marker clusters to keep the map readable.

In [ ]:
# Creating a new column in the spacex_df dataframe to store the marker colours based on the class value

def apply_color(row):
  if row['class'] == 0:
    return 'red'
  else:
    return 'green'

spacex_df['marker_color'] = spacex_df.apply(apply_color, axis = 1)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [ ]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
  marker = folium.Marker(
      location = [record['Lat'], record['Long']],
      icon = folium.Icon(color = 'white', icon_color = record['marker_color'])
  )
  marker_cluster.add_child(marker)

site_map

In [ ]:
# As can be seen for the map above, "KSC LC-39A" has the highest success rate, while the "CCAFS SLC-40" has the highest launch count.

#### Task 3: Calculate the distances between a launch site and its proximities

First add a <code>MousePosition</code> on the map to get coordinate for a mouse point over the map.

In [ ]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
  # approximate radius of earth in Kms
  R = 6373.0

  lat1 = radians(lat1)
  lon1 = radians(lon1)
  lat2 = radians(lat2)
  lon2 = radians(lon2)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))

  distance = R * c
  return distance

In [ ]:
# mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and launch site

# Coordinates of the closest coastline
lnc_lat, lnc_lng = 28.5632, -80.5768
cst_lat, cst_lng = 28.5632, -80.5679

distance_coastline = calculate_distance(lnc_lat, lnc_lng, cst_lat, cst_lng)
distance_coastline

0.8694594933953829

In [ ]:
distance_marker = folium.Marker(
    location = [cst_lat, cst_lng],
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style = "font-size: 12; color: #d35400;"><b>%s</b></div>'%'{:10.2f} KM'.format(distance_coastline)
    )
)

site_map.add_child(distance_marker)
site_map

In [ ]:
# Draw a PolyLine between a launch site to the selected coastline point

lines = folium.PolyLine(locations = [(lnc_lat, lnc_lng), (cst_lat, cst_lng)], weight = 1, tooltip = 'Dist to Coastline')

site_map.add_child(lines)
site_map

In [ ]:
lnc_lat2, lnc_lng2 = 28.5623, -80.5773
ral_lat, ral_lng = 28.5623, -80.5706

distance_railway = calculate_distance(lnc_lat2, lnc_lng2, ral_lat, ral_lng)
distance_railway

0.654542519038652

In [ ]:
railway_marker = folium.Marker(
    location = [ral_lat, ral_lng],
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style = "font-size: 12; color: #d35400;"><b>%s</b></div>'%'{:10.2f} KM'.format(distance_railway)
    )
)

site_map.add_child(railway_marker)
site_map

In [ ]:
line2 = folium.PolyLine(
    locations = [(lnc_lat2, lnc_lng2), (ral_lat, ral_lng)],
    tooltip = 'Distance to railways',
    weight = 1
)

site_map.add_child(line2)
site_map